# Assignment 1

This assignment will involve the creation of a spellchecking system and an evaluation of its performance. You may use the code snippets provided in Python for completing this or you may use the programming language or environment of your choice

Please start by downloading the corpus `holbrook.txt` from Blackboard

The file consists of lines of text, with one sentence per line. Errors in the line are marked with a `|` as follows

    My siter|sister go|goes to Tonbury .
    
In this case the word 'siter' was corrected to 'sister' and the word 'go' was corrected to 'goes'.

In some places in the corpus two words maybe corrected to a single word or one word to a multiple words. This is denoted in the data using underscores e.g.,

    My Mum goes out some_times|sometimes .
    
For the purpose of this assignment you do not need to separate these words, but instead you may treat them like a single token.

*Note: you may use any functions from NLTK to complete the assignment. It should not be necessary to use other libraries and so please consult with us if your solution involves any other external library. If you use any function from NLTK in Task 6 please include a brief description of this function and how it contributes to your solution.*

## Task 1 (10 Marks)

Write a parser that can read all the lines of the file `holbrook.txt` and print out for each line the original (misspelled) text, the corrected text and the indexes of any changes. The indexes refers to the index of the words in the sentence. In the example given, there is only an error in the 10th word and so the list of indexes is [9]. It is not necessary to analyze where the error occurs inside the word.

Then split your data into a test set of 100 lines and a training set.

In [0]:
import nltk as nk
nk.download('all')

In [2]:
# Write your code here
from google.colab import files
import io

uploaded = files.upload()
lines = io.StringIO(uploaded['holbrook.txt'].decode('utf-8')).readlines()


Saving holbrook.txt to holbrook.txt


In [3]:
data = []
original = []
corrected = []

for i in range(len(lines)) :
  lines_array = nk.word_tokenize(lines[i].encode('ascii','ignore'))
  if "|" in lines[i] :
    Error_Data = {
        'original' : list(lines_array),
        'corrected': list(lines_array),
        'Indexes' : []
    }
    for j in Error_Data["original"] :
      if "|" in j :       
        indexes1 = Error_Data["original"].index(j)
        allMisSpelled_words = j.split('|')
        Error_Data['original'][indexes1] = allMisSpelled_words[0]
        Error_Data['corrected'][indexes1] = allMisSpelled_words[1]
        Error_Data['Indexes'].append(indexes1)
       
    data.append(Error_Data)
        
  else :
    Proper_Data = {
        'corrected': lines_array,
        'original': lines_array
    }
    data.append(Proper_Data)
    
print[data[2]]

#assert(data[2] == {
#    'original': ['I', 'have', 'four', 'in', 'my', 'Family', 'Dad', 'Mum', 'and', 'siter', '.'], 
#    'corrected': ['I', 'have', 'four', 'in', 'my', 'Family', 'Dad', 'Mum', 'and', 'sister', '.'], 
#    'indexes': [9]
#})

[{'corrected': ['I', 'have', 'four', 'in', 'my', 'Family', 'Dad', 'Mum', 'and', 'sister', '.'], 'original': ['I', 'have', 'four', 'in', 'my', 'Family', 'Dad', 'Mum', 'and', 'siter', '.'], 'Indexes': [9]}]


The counts and assertions given in the following sections are based on splitting the training and test set as follows

In [0]:
test = data[:100]
train = data[100:]

#Creating Data Sets with original and corrected values: 

trainingCorrectWords = []
for tr in range(len(train)):
  trainingCorrectWords = trainingCorrectWords + train[tr]["corrected"]
# print(len(trainingCorrectWords))

trainingOriginalWords = []
for tr1 in range(len(train)):
  if "original" in train[tr1] : 
    trainingOriginalWords = trainingOriginalWords + train[tr1]['original']

testCorrectWords = []
for tr in range(len(test)):
  testCorrectWords = testCorrectWords + test[tr]["corrected"]
# print(len(testCorrectWords))
  
testOriginalWords = []
for i in range(len(test)):
  if "original" in test[i] : 
    testOriginalWords = testOriginalWords + test[i]['original']
# print(len(testOriginalWords))

## **Task 2** (10 Marks): 
Calculate the frequency (number of occurrences), *ignoring case*, of all words and their unigram probability from the corrected *training* sentences.

*Hint: use `Counter` to implement this so it may be called many times*

In [5]:
from collections import Counter

def unigram(word):
    unigramWords = Counter(w.lower() for w in trainingCorrectWords)
    return unigramWords[word] 
    

def prob(word):
    totalwords = sum(Counter(w.lower() for w in trainingCorrectWords).values())
    prob = float(unigram(word))/totalwords
    return prob
  
print(unigram("me"))

print(prob("me"))
# Test your code with the following
assert(unigram("me")==87)

87
0.00398936170213


## **Task 3** (15 Marks): 
[Edit distance](https://en.wikipedia.org/wiki/Edit_distance) is a method that calculates how similar two strings are to one another by counting the minimum number of operations required to transform one string into the other. There is a built-in implementation in NLTK that works as follows:


In [6]:
from nltk.metrics.distance import edit_distance

# Edit distance returns the number of changes to transform one word to another
print(edit_distance("siter", "sometimes"))

6


Write a function that calculates all words with *minimal* edit distance to the misspelled word. You should do this as follows

1. Collect the set of all unique tokens in `train`
2. Find the minimal edit distance, that is the lowest value for the function `edit_distance` between `token` and a word in `train`
3. Output all unique words in `train` that have this same (minimal) `edit_distance` value

*Do not implement edit distance, use the built-in NLTK function `edit_distance`*

In [8]:
def get_candidates(token):
    Tokens_Train_correct = list(set(trainingCorrectWords))
    dist1 = []
    dist2 = []
    answer = []
    for w in Tokens_Train_correct :
      dist1 = {
          "word" : w,
          "dist" : edit_distance(token, w)
      }
      dist2.append(dist1)

    allDis = []
    for i in dist2 :
      allDis.append(i["dist"])

    MinDis = min(allDis)
    for j in dist2 :
      if j["dist"] == MinDis:
        answer.append(j["word"])
    return answer

get_candidates("minde")    
# Test your code as follows
#assert get_candidates("minde") == ['mine', 'mind']

['mind', 'mine']

## Task 4 (15 Marks):

Write a function that takes a (misspelled) sentence and returns the corrected version of that sentence. The system should scan the sentence for words that are not in the dictionary (set of unique words in the training set) and for each word that is not in the dictionary choose a word in the dictionary that has minimal edit distance and has the highest *unigram probability*. 

*Your solution to this should involve `get_candidates`*


In [9]:
def correct(sentence):
    # Write your code here
    correctSentence = []

    for x in sentence :
      if x in list(set(trainingCorrectWords)) :
        correctSentence.append(x)

      else : 
        a = get_candidates(x)
        wordsAndProb = []
        allWordsProb = []
        selectedWord = []
        for w in a :
              wordsAndProb = {
                  "word" : w,
                  "probability" : prob(w)
              }
              allWordsProb.append(wordsAndProb)
        OnlyProbs = []
        for i in allWordsProb :
          OnlyProbs.append(i["probability"])

        MaxProb = max(OnlyProbs)
        for j in allWordsProb :
          if j["probability"] == MaxProb:
            selectedWord = j["word"]
        correctSentence.append(selectedWord)
    return correctSentence
  
  
correct(["this","whitr","cat"])
#assert(correct(["this","whitr","cat"]) = ['this','white','cat'])   

['this', 'white', 'cat']

## **Task 5** (10 Marks): 
Using the test corpus evaluate the *accuracy* of your method, i.e., how many words from your system's output match the corrected sentence (you should count words that are already spelled correctly and not changed by the system).

In [11]:
test = testOriginalWords
correctedTestData =[]
k = []
counter1 = []
counter2 = []
def accuracy(test):
    # Write your code here
    correctedTestData = correct(test)

    for k in range(len(correctedTestData)):
      if correctedTestData[k] == testCorrectWords[k] :
        counter1.append(k)
      else : 
        counter2.append(k)
    print("Different Words : ", len(counter2))
    print("Same Words : ", len(counter1))
    accuracy = float(len(counter1)*100/(len(counter1) + len(counter2)))
    
    return accuracy

print(accuracy(test))

('Different Words : ', 212)
('Same Words : ', 924)
81.0


## **Task 6 (35 Marks):**

Consider a modification to your algorithm that would improve the accuracy of the algorithm developed in Task 3 and 4

* You may resources beyond those provided here.
* You must **not use the test data** in this task.
* Provide a short text describing what you intend to do and why. 
* Full marks for this section may be obtained without an implementation, but an implementation is preferred.
* Your implementation should not consist of more than 50 lines of code

Please note this task is marked according to: demonstration of knowledge from the lecutures (10), originality and appropriateness of solution (10), completeness of description (10) and technical correctness (5)


**Existing Algorithm’s Explanation:**

---



I have implemented the method in which the provided word are getting compared with the Training Data’s Corrected word set and the words which do not match are navigating into a loop which will find the alternative words from the Training’s Correct Data Set which are at the Minimum distance from the provided word (which is the nearest possible meaning to the word) and then the probability of each alternative word is calculated with the reference to the Unigram of that word from whole data set. And each incorrect/unlisted words are replaced with that max. Probable words having the minimum distance. With Each word being replaced and whole updated data set being prepared, it is compared with the actually corrected data set which is provided by the system and the Accuracy will indicate that up to what extent the Algorithm corrected words match with the actually corrected words and with the above mention implementation I have got accuracy of 81%.

**Modifications to the Existing Algorithm:**

---



While preparing the Algorithm I came up with some of the options and I listed out the methods to increase the Accuracy of the algorithm and one of them I have implemented on the same data set and got the more accurate results. The methods which can contribute to more accurate predictions of the word are: 
1.	**POS Tagging** – Speech Tagging method: Associating a speech tag to each and every word will categorize all the words and the algorithm would be able to replace or find most similar words with more accuracy.

2.	**Bigrams, Trigrams or n-grams** – Converts the unigrams(each word) into a set of 2, 3 or more words and while calculating the probability of any word to occur we need to calculate the Freq. dist. Of the word with context to all the surrounding 2,3, or more words. The method can provide more accurate results but the Time, Memory consumption would lag the system.


3.	**Smoothing Methods, i.e. Good Turing Smoothing** – This method adds the smoothening to the probabilities of each word by adding smoothing component in Numerator and denominator, so if there is any word which has 0 probability will not be neglected. This method is better with large data sets.

4.	**Lesk Algorithm** – Lesk Algorithm takes the sense of the word and returns the most meaningful lemma(Reference of Meaning) of the word and then the distance to nearest words are calculated, and this method is very accurate with the spell corrected data, but for incorrect words data set this method will not be that efficient.  


5.	**Hidden Markov Model(HMM)** – HMM is one of the most efficient method for the sequencing data set, here the data is Sequenced but there are 2 major limitations as we do not know the prob. Of each word having previous references(i.e. Prob. Of Verb after Noun) and here the data set has bunch of sentences and upon completion of each sentence the sequence will break.

6.	**Prediction Models such as word2Vec** – Prediction models uses the same logic to calculate the minimal distances to each word which has been used here.

7.	**Multi-Dimensional Vector Space model** – To calculate the distances more accurately – This is one of the most efficient way to calculate the mean distance but this model plots all the data into multiple dimensions and there are high possibilities that the replaced word would be incorrect one.

8.	**Lemmatization & Normalizing** : Lemmatization is providing the reference of the provided word and Normalizing method will help in getting the most correct word’s reference and then context word from Vector Matrix with POS Reference is taken out which will be the most accurate way but again this method is applicable to the  words which are correct in spelling and we are processing on the data which is misspelled.

9.	**Association Measures by Collection from n-grams **- This method is an extension of n-gram model where data is segregated into n-grams combination and then the association of the words is made to generate the accurate predictions.

**Reasons for Selection of POS Tagging Method:**

---

POS (Part-Of-Speech) tagging is a process where each word is assigned with a grammatical class and categorizing each word under some specific key which is called as tag. In my previous algorithm was just finding the max. probable word with nearest distance from the library(provided Data set) without taking into consideration of the type of word and that’s why “48” which was a Number converted into “4” and name(Noun entity) “NIGEL” converted into “DIED” which has nothing to do with the meaning of the sentences, this occurred into comparatively low Accuracy of the algorithm, while in POS tagging this types of words were provided with the Tags such as for Numbers the tag is “CD”(Cardinal Number) and for Proper Nouns such as Names were given tags of “NNP”. Upon observation I noticed that values of this classes were getting replaced with the another words which could ruin the meaning of the system and would not make any difference even if they are not modified. 

With POS Tagging approach I got the idea of how the whole classification was happening and which classes of Words should be targeted to get more efficient predictions and that’s why I implemented this POS tagging approach which resulted into 89.7% of accuracy which is almost 9% above the previous one while ignoring the “CD” and “NNP” type of tags. The approach behind POS is stochastic approach which includes word frequencies and probability of each word. This approach finds out the most frequently used from the annotated training data for any specific word and uses this tag for the word which is part of Test Data. The POS tagging process is really helpful for earlier analysis and classification of Data.


## **Task 7 (5 Marks):**

Repeat the evaluation (as in Task 5) of your new algorithm and show that it outperforms the algorithm from Task 3 and 4

In [18]:
from nltk import pos_tag
toggle = []

for tr in testOriginalWords:
  x = tr.split()
  toggle = toggle + nk.pos_tag(x)
allwords = [wx for wx, px in toggle]

correct_counter = []
incorrect_counter = []
def accuracy1(x):
    # Write your code here
    check_correct_words = correct(allwords)
    for cword in range(len(check_correct_words)):      
      if ((toggle[cword][1] == 'CD') | (toggle[cword][1] == 'NNP') | (toggle[cword][0] == testCorrectWords[cword])):
        correct_counter.append(toggle[cword][0])  
      else:
        incorrect_counter.append(toggle[cword][0])
    calc_accuracy = float(len(correct_counter))/len(check_correct_words)*100
    print("TRUE ",len(correct_counter))
    print("FALSE ",len(incorrect_counter))
 
    return calc_accuracy

print(accuracy1("asd"))

('TRUE ', 1019)
('FALSE ', 117)
89.7007042254
